# Using Sherlock out-of-the-box
This notebook shows how to predict a semantic type for a given table column.
The steps are basically:
- Extract features from a column.
- Initialize Sherlock.
- Make a prediction for the feature representation of the column.

In [1]:
import numpy as np
import pandas as pd
import pyarrow as pa

from sherlock import helpers
from sherlock.deploy.model import SherlockModel
from sherlock.functional import extract_features_to_csv
from sherlock.features.paragraph_vectors import initialise_pretrained_model, initialise_nltk
from sherlock.features.preprocessing import (
    extract_features,
    convert_string_lists_to_lists,
    prepare_feature_extraction,
    load_parquet_values,
)
from sherlock.features.word_embeddings import initialise_word_embeddings

In [2]:
%env PYTHONHASHSEED

'13'

## Extract features

In [3]:
prepare_feature_extraction()
initialise_word_embeddings()
initialise_pretrained_model(400)
initialise_nltk()

Preparing feature extraction by downloading 2 files:
        
 ../sherlock/features/glove.6B.50d.txt and 
 ../sherlock/features/par_vec_trained_400.pkl.docvecs.vectors_docs.npy.
        
All files for extracting word and paragraph embeddings are present.
Initialising word embeddings
Initialise Word Embeddings process took 0:00:05.275641 seconds.
Initialise Doc2Vec Model, 400 dim, process took 0:00:02.651327 seconds. (filename = ../sherlock/features/par_vec_trained_400.pkl)
Initialised NLTK, process took 0:00:00.183191 seconds.


[nltk_data] Downloading package punkt to /Users/madelon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/madelon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
data = pd.Series([["madelon hulsebos", "lute something", "anna anotherhing"], ["Binnenkant 2", "Binnenkant 3 1011BH"]], name="values")

In [5]:
data

0    [madelon hulsebos, lute something, anna anothe...
1                  [Binnenkant 2, Binnenkant 3 1011BH]
Name: values, dtype: object

In [6]:
pd.DataFrame(data).astype(str).to_parquet("../data/data/raw/temporary.parquet")

### Issue 1:  `extract_features`
The below code yields incorrect features due to `,` (as a character features) being read as a separator in the "read_csv".

In [7]:
extract_features(
    "../temporary.csv",
    data
)
feature_vector = pd.read_csv("../temporary.csv", dtype=np.float32)

Extracting Features: 100%|██████████| 2/2 [00:00<00:00, 134.55it/s]

Exporting 1588 column features


In [8]:
feature_vector

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,-3.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,0.0,0.5,0.25,0.0,1.0,0.5,1.0,-2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
feature_vector.shape

(2, 1598)

In [10]:
feature_vector.columns.str.startswith('n_[,').sum()

0

Problem: 1598 features due to the `n_[,]-agg-any` being split due to the csv dataframe dump

### Issue 2: `extract_features_to_csv`

In [11]:
values = load_parquet_values("../data/data/raw/temporary.parquet")

In [12]:
values[0]

<pyarrow.StringScalar: "['madelon hulsebos', 'lute something', 'anna anotherhing']">

In [13]:
extract_features_to_csv("../data/data/processed/temporary.csv", values)

Starting ../data/data/processed/temporary.csv at 2022-02-11 08:52:49.908673. Rows=2, using 8 CPU cores
[
  [
    "['madelon hulsebos', 'lute something', 'anna anotherhing']",
    "['Binnenkant 2', 'Binnenkant 3 1011BH']"
  ]
]
None


Error: iterable expected, not NoneType

### Reproducing the problem with the original file (works in notebook `01-data-processing`)

In [15]:
values = load_parquet_values("../data/data/raw/test_values.parquet")

extract_features_to_csv("../data/data/processed/temporary.csv", values)

values = None

Starting ../data/data/processed/temporary.csv at 2022-02-11 08:56:21.898853. Rows=137353, using 8 CPU cores
[
  [
    "['Central Missouri', 'unattached', 'unattached', 'Kansas State University', 'unattached', 'North Dakota State', 'Nike']",
    "[95, 100, 95, 89, 84, 91, 88, 94, 75, 78, 90, 84, 90, 76, 93, 70, 80, 80, 82]",
    "['Katie Crews', 'Christian Hiraldo', 'Alex Estrada', 'Fredy Peltroche', 'Xavier Perez', 'Gustavo Larrosa', 'Jose Montano', 'Angel Cruz (7)', 'J Acosta']",
    "['Christian', 'Non-Christian', 'Unreported', 'Jewish', 'Atheists']",
    "['AAF-McQuay Canada Inc.', 'AAF-McQuay Canada Inc.', 'Ability Janitorial Services Ltd.', 'Acart Communications Inc.', 'Accu-lift Flooring Systems', 'Accurate Point Construction Ltd.  ', 'Acklands-Grainger Inc.', 'Acme Future Security Controls Inc.', 'ACMG Management Inc.', 'Advanced Chippers Technologies Inc.', 'Advanced Chippewa Technologies Inc.', 'Advanced Chippewa Technologies Inc.', 'Advanced Chippewa Technologies Inc.', 'Adva

None


Error: iterable expected, not NoneType

In [30]:
feature_vector = pd.read_csv("../data/data/processed/temporary.csv")

Problem 2: extracting features to a csv file does not work in this notebook, while it works in 01-data-processing (see below). This seems odd as the environment is the same, the data is the same, imports and preprocessing steps are the same.

## Initialize Sherlock.

In [27]:
model = SherlockModel()
model.initialize_model_from_json(with_weights=True);

## Predict semantic type for column


TODO: there is a problem with the featurization with `extract_features`.

Will be completed soon.

In [30]:
predicted_labels = model.predict(feature_vector, "sherlock")

KeyError: "['n_[,]-agg-all', 'n_[,]-agg-median', 'n_[,]-agg-max', 'n_[,]-agg-sum', 'n_[,]-agg-any', 'n_[,]-agg-mean', 'n_[,]-agg-kurtosis', 'n_[,]-agg-skewness', 'n_[,]-agg-min', 'n_[,]-agg-var'] not in index"